<a href="https://colab.research.google.com/github/azizamirsaidova/abstractive_text_summarization/blob/main/Fine_tune_simplet5_absractive_text_summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np 
import pandas as pd 

In [ ]:
trainning_df = pd.read_csv('train.csv', engine='python', error_bad_lines=False)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
MAX_LEN = 300
SUMMARY_LEN = 100
TRAINNING_SIZE = 2000

In [ ]:
trainning_df = trainning_df.iloc[0:TRAINNING_SIZE,:].copy()

trainning_article_ls = list(trainning_df['article'])
trainning_highlight_ls = list(trainning_df['highlights'])

del trainning_df

In [ ]:
from random import sample

for index in sample(list(np.arange(len(trainning_article_ls))),3):
    print('Original Text : ')
    print(trainning_article_ls[index])

    print('\n\nSummary Text : ')
    print(trainning_highlight_ls[index])
    
    print('===========================================================================================================\n\n')

Original Text : 
By . Suzannah Hills . PUBLISHED: . 07:37 EST, 19 January 2013 . | . UPDATED: . 16:27 EST, 19 January 2013 . From a perfectly sculpted snowman riding a bicycle to a dog lover taking his Alaskan malamutes for their first ever sledding experience in the unlikely location of Barnes Park in Sunderland, Tyne & Wear - your pics are helping us capture Britain as it is swept by snow. Whether . it be a snow-topped castle, or a blizzard in full force, we are asking . our readers across the UK to send in their best pictures, capturing the . effects of 2013's first cold snap. Readers from all corners of the UK - from Plymouth to Middlesbrough - have already been sending their snow-tinged pictures. If . it is beautiful scenery you have captured or just somebody having fun, . help us share the experience with readers across the country. Are you out frolicking in the snow? Email all your pictures to mailonlinepictures@dailymail.co.uk . Going the extra mile: Lizzy Eddy sent in this pic

In [ ]:
print(len(trainning_article_ls))
print(len(trainning_highlight_ls))

5000
5000


In [ ]:
df = pd.DataFrame(columns=['target_text','source_text'])
df['target_text'] = trainning_highlight_ls
df['source_text'] = ['summarize: '+item for item in trainning_article_ls]

In [ ]:
df.head()

,target_text,source_text
0,"Bishop John Folda, of North Dakota, is taking ...",summarize: By . Associated Press . PUBLISHED: ...
1,Criminal complaint: Cop used his role to help ...,summarize: (CNN) -- Ralph Mata was an internal...
2,"Craig Eccleston-Todd, 27, had drunk at least t...",summarize: A drunk driver who killed a young w...
3,Nina dos Santos says Europe must be ready to a...,summarize: (CNN) -- With a breezy sweep of his...
4,Fleetwood top of League One after 2-0 win at S...,summarize: Fleetwood are the only team still t...


https://www.kaggle.com/nulldata/training-t5-models-made-easy-with-simplet5#Model-Training

In [ ]:
!pip install tf-estimator-nightly==2.8.0.dev2021122109
!pip install folium==0.2.1
!pip install simplet5 -q

In [ ]:
from simplet5 import SimpleT5

model = SimpleT5()
model.from_pretrained(model_type="t5", model_name="t5-base")

Global seed set to 42


In [ ]:
MAX_EPOCHS = 5

In [ ]:
model.train(train_df=df[0:(int)(0.3*TRAINNING_SIZE)],
            eval_df=df[(int)(0.3*TRAINNING_SIZE):TRAINNING_SIZE], 
            source_max_token_len=MAX_LEN, 
            target_max_token_len=SUMMARY_LEN, 
            batch_size=2, max_epochs=MAX_EPOCHS, use_gpu=True)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 42


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [ ]:
import os
model_path = ''
rootdir = 'outputs/'
for it in os.scandir(rootdir):
    if it.is_dir():
        if 'simplet5-epoch-'+(str)(MAX_EPOCHS-1) in it.path:
            model_path = it.path
            print(model_path)

outputs/simplet5-epoch-4-train-loss-0.8108-val-loss-1.739


In [ ]:
model.load_model("t5","./"+model_path, use_gpu=True)

In [ ]:
import torch
import json 
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config

no_tune_model = T5ForConditionalGeneration.from_pretrained('t5-base')
tokenizer = T5Tokenizer.from_pretrained('t5-base')

device = torch.device('cpu')

In [ ]:
from random import sample
from rouge import Rouge


for index in sample(list(np.arange(len(trainning_article_ls))),5):
    print('Original Text : ')
    print(trainning_article_ls[index])

    Original_article = trainning_article_ls[index]

    print('\n\nSummary Text : ')
    print(trainning_highlight_ls[index])

    Original_Summary = trainning_highlight_ls[index]

    print('\n\nFine tuned Predicted Summary Text : ')
    print(model.predict(trainning_article_ls[index]))

    Predicted_Summary = trainning_article_ls[index]

    rouge = Rouge()

    print("-----------------")

    print(rouge.get_scores(Predicted_Summary, Original_Summary))

    print("-----------------")
    
    print('\n\nNot Fine tuned Predicted Summary Text : ')
    preprocess_text = trainning_article_ls[index].strip().replace("\n","")
    t5_prepared_Text = "summarize: "+preprocess_text
    tokenized_text = tokenizer.encode(t5_prepared_Text, return_tensors="pt").to(device)
    summary_ids = no_tune_model.generate(tokenized_text,
                                        num_beams=4,
                                        no_repeat_ngram_size=2,
                                        min_length=30,
                                        max_length=SUMMARY_LEN,
                                        early_stopping=True)
    output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    print(output)
    print('===========================================================================================================\n\n')

Original Text : 
By . Craig Hope . Follow @CraigHope_DM . After a 110-year wait, Hull City's European adventure very nearly lasted just seven days until Sone Aluko intervened to prolong their stay. But on the evidence of this lethargic and naïve performance, Steve Bruce's side will have to re-emerge with a newfound vigour and maturity when they face a final qualifier later this month. For now, though, relief is the overriding emotion on Humberside. VIDEO Scroll down for Steve Bruce is delighted for the supporters after a late win . Winner: Sone Aluko celebrates scoring the pivotal goal in the 2-1 win for Hull City against AS Trencin . Technique: The substitute adjusted well to fire home the winner at the KC Stadium . Delight: Hull City's European adventure can now continue . Crucial: Ahmed Elmohamady nods home an equaliser for Hull City against AS Trencin . Dream start: Tomas Malec gave the visitors a shock lead early on in the UEFA Europa League qualifier . HULL CITY:  McGregor 6; Bru

In [ ]:
#!pip install rouge_score
!pip install rouge
